# Convolutional Neural Networks

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers, models, optimizers, losses, metrics, datasets

In [3]:
BATCH_SIZE = 128
ETA = 1e-3
EPOCHS = 10

다음 코드는 tensorflow에 내장되어 있는 MNIST 데이터셋을 불러오는 코드입니다. 이때, `trainset`, `testset`은 다음과 같은 모양이 됩니다.
```python
trainset = (x_train, y_train)
testset = (x_test, y_test)
```

In [4]:
trainset, testset = datasets.mnist.load_data()

이것을 tensorflow dataset `tf.data.Dataset` 객체로 만들어줄 것입니다. `trainset`, `testset`은 numpy객체이고, `tf.data.Dataset`은 `.from_tensor_slices`라는 메소드를 제공하여 numpy array를 `tf.data.Dataset`으로 변환하는 기능을 제공합니다.

`tf.data.Dataset.from_tensor_slices()`는 튜플을 입력으로 받는데, 튜플은 `(data_array, label_array)`의 형태이어야 합니다. `mnist.load_data()`메소드가 이미 그런 형태로 `trainset`, `testset`을 생성해 주었으므로, 그냥 그대로 `.from_tensor_slices`에 인자로 넘겨주면 됩니다.

In [5]:
trainloader = tf.data.Dataset.from_tensor_slices(trainset).batch(BATCH_SIZE).shuffle(4096)
testloader = tf.data.Dataset.from_tensor_slices(testset).batch(BATCH_SIZE)

여기서, `.batch(BATCH_SIZE).shuffle(4096)`라는 문구가 추가되었네요. 이것을 알기 이전에, `tf.data.Dataset`을 좀 더 알 필요가 있는데요, `tf.data.Dataset`은 python의 generator입니다.
즉, `iter()`로 감쌀 수 있고, `next()`함수로 어떤 원소를 generate할 수 있는 객체입니다. `tf.data.Dataset`은 `next()`를 한 번 호출할 때마다 데이터 샘플 1개를 기본적으로 반환합니다. 하지만, `.batch(BATCH_SIZE)`라는 메소드를 호출해주면, `next()`를 한 번 호출해 줄 때마다 `BATCH_SIZE`개 만큼의 데이터를 반환해줍니다.

`.shuffle(4096)`의 경우, 1번의 epoch을 돌때마다 `tf.data.Dataset`안의 데이터를 셔플해주라는 의미입니다. `4096`은 그냥 버퍼크기로, 셔플의 속도에 아주 조금 차이가 있는 것 같습니다.

## Implementation

이제, CNN을 구현해보겠습니다.

In [6]:
model = models.Sequential([
    layers.Conv2D(16, (3, 3), strides=(1, 1), padding="SAME"),
    layers.BatchNormalization(),
    layers.Activation(tf.nn.leaky_relu),
    
    layers.MaxPooling2D((2, 2), strides=(2, 2), padding="VALID"),

    layers.Conv2D(32, (3, 3), strides=(1, 1), padding="SAME"),
    layers.BatchNormalization(),
    layers.Activation(tf.nn.leaky_relu),
    
    layers.MaxPooling2D((2, 2), strides=(2, 2), padding="VALID"),
    
    layers.Flatten(),
    
    layers.Dense(32),
    layers.Activation(tf.nn.leaky_relu),
    layers.Dropout(0.5),

    layers.Dense(10),
    layers.Activation(tf.nn.softmax)
])

위 코드는 CNN의 모델을 구현한 것인데요, 코드를 하나하나 뜯어보겠습니다.

```python
model = models.Sequential([
    ...
])
```

이 구조는 이전 fully connected network에서도 보셨을 것입니다. `models.Sequential`은 tensorflow(keras)의 **모델**로, 특히, 내부 각 레이어들을 sequential하게 실행시키는 모델입니다.
위 코드는 model이라는 변수에 `models.Sequential`객체를 생성하여 대입하는 코드입니다.

`models.Sequential`의 인자로는 리스트가 들어가는데요, 이때 리스트는 레이어의 리스트가 됩니다. 이 리스트 안을 하나하나 보겠습니다.

```python
layers.Conv2D(16, (3, 3), strides=(1, 1), padding="SAME"),
```

위 코드는 convolution layer를 생성하는 코드입니다. 이때, output channel은 16입니다. 즉, 이 convolution layer의 filter 수는 16개입니다. filter의 크기는 (3, 3)입니다. 즉, 세로3, 가로3 이죠. stride는 세로,가로 각각 1씩 줍니다(`strides=(1, 1)`). padding의 경우 `"SAME"`을 주었는데, padding에 들어갈 수 있는 것은 딱 두가지입니다.
- `"SAME"`
- `"VALID`

`"SAME"`은 `strides=(1, 1)`을 기준으로 했을 때, convolution의 결과로 height, width가 똑같이 유지되도록 입력 이미지에 알아서 패딩해줍니다. 예를 들어서, filter 크기가 (3, 3)이라면, stride가 (1, 1)일때, convolution의 결과로 height, width는 2씩 줄어들 것입니다. 하지만 `padding="SAME"`을 하게 되면, height, width에 2씩 패딩을 해 줍니다(`padding=1` - 상하좌우이므로 1을 해주면 height width가 2씩 늘어남)

`"VALID"`는 패딩을 하지 말라는 것입니다.

```python
layers.BatchNormalization(),
```

이것은 batch normalization 레이어를 생성하는 코드입니다.

```python
layers.Activation(tf.nn.leaky_relu),
```

이것은 activation을 적용하라는 것인데, 이때, activation함수로 leaky relu를 사용한 모습입니다. 주의할 점은 `Activation`의 인자로 `tf.nn.leaky_relu()`를 넘기면 안되고 `tf.nn.leaky_relu`를 넘겨야 합니다. 함수 자체를 넘겨야 한다는 것입니다.

보통 위 세가지를 sequential하게 배치한 것을 convolution 1 block이라고 부르는 듯 합니다. 즉, 다음은 하나의 convolution block입니다.

```python
layers.Conv2D(16, (3, 3), strides=(1, 1), padding="SAME"),
layers.BatchNormalization(),
layers.Activation(tf.nn.leaky_relu),
```

다음 코드는 max pooling 레이어를 하나 생성하는 코드입니다.

```python
layers.MaxPooling2D((2, 2), strides=(2, 2), padding="VALID"),
```

이때, 첫 번째 인자로는 pooling size, 보통 (2, 2)이죠. stride도 보통 (2, 2)를 줍니다. 그래야 아웃풋의 가로세로 크기가 입력의 절반으로 줄어듭니다. 마지막으로 `padding="VALID"`를 넣어줍니다. 사실 `padding="SAME"`을 줘도 상관없습니다. 결과는 똑같이 절반으로 줄어듭니다.

이제, loss와 optimizer, 그리고 출력을 위한 보조 객체를 생성해보겠습니다.

In [7]:
# loss 함수: sparse categorical cross entropy
criterion = losses.SparseCategoricalCrossentropy()

# optimizer: adam optimizer
optimizer = optimizers.Adam(learning_rate=ETA)

# 나중에 print에 쓸려고 loss를 축적하는데 사용하는 객체
train_loss_obj = metrics.SparseCategoricalCrossentropy()
test_loss_obj = metrics.SparseCategoricalCrossentropy()

# 나중에 print에 쓸려고 accuracy를 축적하는데 사용하는 객체
train_acc_obj = metrics.SparseCategoricalAccuracy()
test_acc_obj = metrics.SparseCategoricalAccuracy()

다음은, 배치에 대한 학습 함수와 테스트함수를 정의합니다.

In [8]:
def train_step(x, y):
    """
    1 배치에 대한 학습
    
    Arguments:
    ----------
    - x: image batch (batch_size, 28, 28, 1)
    - y: label (batch_size,)
    """
    
    with tf.GradientTape() as tape:
        # forward propagation
        ps = model(x)
        # compute loss
        loss = criterion(y, ps)
        
    # compute gradient with respect to trainable weights
    grads = tape.gradient(loss, model.trainable_variables)
    # update weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    # 나중에 loss를 출력하기 위해 모아주는 것입니다.
    train_loss_obj.update_state(y, ps)
    train_acc_obj.update_state(y, ps)

In [9]:
def test_step(x, y):
    """
    1 배치에 대한 test(학습없이 accuracy 및 loss만 계산)
    
    Arguments:
    ----------
    - x: 이미지 배치 (batch_size, 28, 28, 1)
    - y: 라벨 (batch_size,)
    """
    
    # forward propagation
    ps = model(x)
    # compute loss
    loss = criterion(y, ps)
    
    # 테스트 loss와 accuracy를 계산해서 기록
    test_loss_obj.update_state(y, ps)
    test_acc_obj.update_state(y, ps)

이제, 학습을 진행할 차례입니다.

In [10]:
def train():
    """
    학습
    """
    
    # Epochs 만큼 데이터셋을 반복 학습
    for e in range(EPOCHS):
        
        # 한 번에 배치만큼 데이터를 trainset에서 뽑아서 학습
        for x, y in trainloader:
            x = tf.cast(x, tf.float32)/256
            x = tf.expand_dims(x, axis=-1)
            train_step(x, y)
            
        # 한 번에 배치만큼 데이터를 testset에서 뽑아서 테스트
        for x, y in testloader:
            x = tf.cast(x, tf.float32)/256
            x = tf.expand_dims(x, axis=-1)
            test_step(x, y)
            
        # 기록해두었던 loss들을 얻어옴. 평균은 자동으로 내줌
        train_loss = train_loss_obj.result()
        test_loss = test_loss_obj.result()
        
        # 기록해두었던 accuracy들을 얻어옴. 평균은 자동으로 내줌
        train_acc = train_acc_obj.result()
        test_acc = test_acc_obj.result()
        
        # 출력
        print(f"Epochs {e+1}/{EPOCHS}, train loss: {train_loss:.8f}, train acc: {train_acc:.4f}, test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")
        
        # 기록들을 모두 삭제
        train_loss_obj.reset_states()
        test_loss_obj.reset_states()
        train_acc_obj.reset_states()
        test_acc_obj.reset_states()

In [11]:
train()

Epochs 1/10, train loss: 0.28369382, train acc: 0.9161, test loss: 0.09698427, test acc: 0.9697
Epochs 2/10, train loss: 0.07899778, train acc: 0.9760, test loss: 0.06216047, test acc: 0.9779
Epochs 3/10, train loss: 0.05742297, train acc: 0.9819, test loss: 0.05329134, test acc: 0.9807
Epochs 4/10, train loss: 0.04477991, train acc: 0.9864, test loss: 0.05657598, test acc: 0.9816
Epochs 5/10, train loss: 0.03797453, train acc: 0.9883, test loss: 0.05099284, test acc: 0.9828
Epochs 6/10, train loss: 0.03162078, train acc: 0.9900, test loss: 0.03605920, test acc: 0.9879
Epochs 7/10, train loss: 0.02724693, train acc: 0.9916, test loss: 0.04116359, test acc: 0.9865
Epochs 8/10, train loss: 0.02431102, train acc: 0.9922, test loss: 0.03554447, test acc: 0.9888
Epochs 9/10, train loss: 0.02051357, train acc: 0.9935, test loss: 0.04146039, test acc: 0.9862
Epochs 10/10, train loss: 0.01640139, train acc: 0.9947, test loss: 0.04217651, test acc: 0.9869
